# Mise en place d'un système de multiplexage de 3 stations météo

Importation des librairies générales et importation des binaires pour les différents capteurs utilisés dans les montages.

In [1]:
import mraa, upm
import _thread, time, math

from upm import pyupm_mcp9808 as mcp
from upm import pyupm_bmp280 as bme
from upm import pyupm_tsl2561 as tsl
from upm import pyupm_htu21d as htud
from upm import pyupm_mpl3115a2 as mpl
from upm import pyupm_si1132 as si
from upm import pyupm_tca9548a as multi

## Déclaration des capteurs

Déclaration des adresses des capteurs et des constantes utilisées pour les mesures.

In [2]:
# Constant declarations
#----------------------
# i2c bus 1 on RaspBerry Pi
bus1 = 0
mcpAddr = 0x18
bmeAddr = 0x77
bmpAddr = 0x76
tslAddr = 0x39

localAlt = 156.5
UPPER_TEMP = 0x02
LOWER_TEMP = 0x03
CRIT_TEMP = 0x04

## Ouverture du canal 4 du multiplexeur

On déclare le multiplexeur et on ouvre le canal 4 où sont connectés les capteurs.

In [3]:
# Select channel 4 on the multiplexer 
multiplex1 = multi.TCA9548A(bus1, multi.TCA9548A_DEFAULT_ADDR)
multiplex1.selSlave(multi.TCA9548A_DEFAULT_ADDR, multi.TCA9548A.C4)

In [4]:
# create sensors on the bus 1
temp = mcp.MCP9808(bus1)
hygro = bme.BME280(bus1,bmeAddr)
luxtsl = tsl.TSL2561(bus1, tslAddr)

In [5]:
# Sensor Functions
#---------
# bme 280
#****************
def get_bmSensor(thisSensor) :
    name = thisSensor.getModuleName ()
    bmpId = thisSensor.getChipID ()
    print ('capteur : %s\t'%name + 'Id : %d\n'%bmpId)
    return

def reset_bmSensor(thisSensor) :
    thisSensor.reset()
    return

def mes_bmTPA(thisSensor) :
    thisSensor.update ()
    bmTemp = thisSensor.getTemperature ()
    bmPress = thisSensor.getPressure () /100.0
    bmAlt = thisSensor.getAltitude ()
    return(bmTemp, bmPress, bmAlt)

def mes_bmeHum(thisSensor) :
    thisSensor.update ()
    bmeHum = thisSensor.getHumidity ()
    return(bmeHum)

def convertSeaLevel (m_Pressure,m_Temperature,m_alt):
#    s_Pressure = m_Pressure / math.pow(1.0 - localAlt/44330, 5.255)
    s_Pressure = m_Pressure * math.pow((1 - 0.0065 * m_alt / ( m_Temperature + 0.0065 * m_alt + 273.15)), -5.257) 
#    s_Pressure = m_Pressure / math.pow((1 - 0.0065 * m_alt / m_Temperature), 5.255 )
    return s_Pressure

# MCP9808
#********
def get_mcpSensor(thisSensor):
    unity = thisSensor.isCelsius()
    upper = thisSensor.getMonitorReg (UPPER_TEMP)
    lower = thisSensor.getMonitorReg (LOWER_TEMP)
    critic = thisSensor.getMonitorReg (CRIT_TEMP)
    hyst = thisSensor.getHysteresis ()
    resol = thisSensor.getResolution ()
    idManuf = thisSensor.getManufacturer ()
    mcpId = thisSensor.getDevicedId ()
    if unity :
        print ('We are measuring in Celsius')
    else :
        print ('We are measuring in Fahrenheit')
    print ('\tLower T° : %.3f\n'%lower + '\tUpperT° : %.3f\n'%upper + '\tCritical T° : %.3f\n'%critic)
    print ('Hysteresis value : %.3f'%hyst)
    print ('Resolution : %.4f'%resol)
    print ('Manufacturer Id : %d'%idManuf)
    print ('Device Id : %d\n'%mcpId)
    return

def mes_mcpTemp(thisSensor):
    mcpTemp = thisSensor.getTemp ()
    return (mcpTemp)

# TSL2561
#*******
def mes_tslLux(thisSensor) :
    lux = thisSensor.getLux ()
    return(lux)

def input_thread(CheckInput):
    check = input()
    CheckInput.append(check)
    return

def init_Thread () :
    CheckInput = []
    _thread.start_new_thread(input_thread, (CheckInput,))
    print ('Instructions :  \t"stop" pour arrêter les mesures.\n')
    print ('\t\t"resetBME" pour redémarrer le capteur BME280')
    return (CheckInput)


In [6]:
# Main function
###############
def measureMeteoParams():
    nbmes = 0
    while True :
        option1 = input('Instructions :  \t"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.\n--> ')
        if option1 == 'exit' :
            break
        elif option1 == 'start' :
            break
# Main loop
    if option1.strip() == 'start' :
        CheckInput = init_Thread()
        while True :
            if CheckInput == ['stop']: 
                print ('Arrêt demandé.')
                break
            elif CheckInput != [] :
                print ('Commande inconnue : "%s"\n'%CheckInput)
                thread.start_new_thread(input_thread, (CheckInput,))
                del CheckInput[:]
            else :
                time.sleep (0.5)
            tempHR = mes_mcpTemp(temp)
            hygroBME = mes_bmeHum(hygro) 
            [tempBME,pressBME,altBME] = mes_bmTPA(hygro)
            seaLevelPress = convertSeaLevel (pressBME,tempBME,localAlt)
            lux = mes_tslLux(luxtsl)
            if nbmes == 0 :
                print ('Int. Lum. : \tHumid(BME) : \tTemp(MCP) : \tTemp(BME) :  \tAlt(BME) : \tPress(BME) : \tPress(SealevelBME) : ' )
                nbmes += 1
            elif nbmes == 9 :
                nbmes = 0
            else :
                nbmes += 1
            print ('%.2f\t\t' %lux + '%2.1f\t\t' %hygroBME + '%.4f °C\t' %tempHR +  '%.2f °C\t' %tempBME + '%.2f m\t\t' %altBME  + '%.2f hPa\t\t' %pressBME + '%.2f hPa\t' %seaLevelPress)

            time.sleep (5)
    else :
        print ('Bye...')
    return

measureMeteoParams()


Instructions :  	"start" pour démarrer les mesures "check" pour la maintenance "exit" pour sortir.
--> start
Instructions :  	"stop" pour arrêter les mesures.

		"resetBME" pour redémarrer le capteur BME280
Int. Lum. : 	Humid(BME) : 	Temp(MCP) : 	Temp(BME) :  	Alt(BME) : 	Press(BME) : 	Press(SealevelBME) : 
12.00		38.6		23.3750 °C	25.04 °C	80.66 m		1003.59 hPa		1021.72 hPa	
12.00		38.6		23.3750 °C	25.04 °C	80.68 m		1003.59 hPa		1021.72 hPa	
stop
Arrêt demandé.


In [15]:
# Close Channel 4
multiplex1.closeSlaves(multi.TCA9548A_DEFAULT_ADDR)

In [16]:
# Select channel 2 on the multiplexer 
multiplex2 = multi.TCA9548A(bus1, multi.TCA9548A_DEFAULT_ADDR)
multiplex2.selSlave(multi.TCA9548A_DEFAULT_ADDR, multi.TCA9548A.C2)

In [17]:
# create sensors on the bus 1
tphyg = htud.HTU21D(bus1)
bmp = bme.BMP280(bus1)
ppress = mpl.MPL3115A2(bus1)

In [18]:
name = bmp.getModuleName ()
bmpId = bmp.getChipID ()
print ('capteur : %s\t'%name + 'Id : %d'%bmpId)

capteur : BMP280	Id : 88


In [19]:
bmp.update ()
bmTemp = bmp.getTemperature ()
bmPress = bmp.getPressure () /100.0
bmAlt = bmp.getAltitude ()
print ('Temp(BMP) :  \tAlt(BMP) : \tPress(BMP) : ' )
print ('%.4f °C\t' %bmTemp + '%.2f m\t\t' %bmAlt  + '%.2f hPa' %bmPress)

Temp(BMP) :  	Alt(BMP) : 	Press(BMP) : 
24.4300 °C	79.84 m		1003.69 hPa


In [24]:
# Close Channel 4
multiplex2.closeSlaves(multi.TCA9548A_DEFAULT_ADDR)

In [25]:
#channels = {multi.TCA9548A.RESET,multi.TCA9548A.RESET,multi.TCA9548A.C2,multi.TCA9548A.RESET,multi.TCA9548A.C4,multi.TCA9548A.RESET,multi.TCA9548A.RESET,multi.TCA9548A.RESET}
multiplex2 = multi.TCA9548A(bus1, multi.TCA9548A_DEFAULT_ADDR)
multiplex2.selMultSlaves(multi.TCA9548A_DEFAULT_ADDR, multi.TCA9548A.RESET, multi.TCA9548A.RESET, multi.TCA9548A.C2, multi.TCA9548A.RESET, multi.TCA9548A.C4, multi.TCA9548A.RESET, multi.TCA9548A.RESET, multi.TCA9548A.RESET)

In [22]:
tempHR = mes_mcpTemp(temp)
hygroBME = mes_bmeHum(hygro) 
[tempBME,pressBME,altBME] = mes_bmTPA(hygro)
seaLevelPress = convertSeaLevel (pressBME,tempBME,localAlt)
lux = mes_tslLux(luxtsl)
print ('Int. Lum. : \tHumid(BME) : \tTemp(MCP) : \tTemp(BME) :  \tAlt(BME) : \tPress(BME) : \tPress(SealevelBME) : ' )
print ('%.2f\t\t' %lux + '%2.1f\t\t' %hygroBME + '%.4f °C\t' %tempHR +  '%.2f °C\t' %tempBME + '%.2f m\t\t' %altBME  + '%.2f hPa\t\t' %pressBME + '%.2f hPa\t' %seaLevelPress)
bmp.update ()
bmTemp = bmp.getTemperature ()
bmPress = bmp.getPressure () /100.0
bmAlt = bmp.getAltitude ()
print ('Temp(BMP) :  \tAlt(BMP) : \tPress(BMP) : ' )
print ('%.4f °C\t' %bmTemp + '%.2f m\t\t' %bmAlt  + '%.2f hPa' %bmPress)


Int. Lum. : 	Humid(BME) : 	Temp(MCP) : 	Temp(BME) :  	Alt(BME) : 	Press(BME) : 	Press(SealevelBME) : 
12.00		0.0		23.4375 °C	-139.10 °C	2370.67 m		758.95 hPa		789.72 hPa	
Temp(BMP) :  	Alt(BMP) : 	Press(BMP) : 
-139.7200 °C	1662.41 m		828.77 hPa


In [23]:
# Close all channels
multiplex2.closeSlaves(multi.TCA9548A_DEFAULT_ADDR)